---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [7]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [13]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    
    data = pd.read_table('university_towns.txt', header=None)

    state_pattern = re.compile("(\S+)\[edit\]")

    state_list = []
    city_list = []

    for item in data[0]:
        if state_pattern.search(item):
            current_state = item
            continue
        state_list.append(current_state)
        city_list.append(item)

    list_of_towns = pd.DataFrame({'State': state_list,
                                  'RegionName': city_list})

    list_of_towns = list_of_towns.replace('\[edit\]| \(.*', '', regex=True)

    
    return list_of_towns


In [90]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4, 5, 6])
    gdp.columns= ['CYQ', 'GDP in billions of current dollars',
                  'GDP in billions of chained 2009 dollars']

    # extracting quarters from 2000 and on
    gdp = gdp[gdp['CYQ'].str[:4].astype(int)>= 2000]

    gdp['difference_2009'] = gdp['GDP in billions of chained 2009 dollars'].diff()
    gdp['signal'] = gdp['difference_2009'] < 0
    gdp['signal'] = gdp['signal'].apply(lambda z:1 if z else 0)


    # Boolean indexers for recession start and stops.
    rec_start = (gdp['signal'] == 1) & (gdp['signal'].shift(-1) == 1)
    rec_end = (gdp['signal'] == 0) & (gdp['signal'].shift(-1) == 0)

    # Mark the recession start/stops as True/False.
    gdp.loc[rec_start, 'recession'] = True
    gdp.loc[rec_end, 'recession'] = False

    # Forward fill the recession column with the last known Boolean.
    # Fill any NaN's as False (i.e. locations before the first start/stop).
    gdp['recession'] = gdp['recession'].ffill().fillna(False)
    
    return gdp[gdp['recession'] == True]['CYQ'].min()


'2008q3'

In [112]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4, 5, 6])
    gdp.columns= ['CYQ', 'GDP in billions of current dollars',
                  'GDP in billions of chained 2009 dollars']

    # extracting quarters from 2000 and on
    gdp = gdp[gdp['CYQ'].str[:4].astype(int)>= 2000]

    gdp['difference_2009'] = gdp['GDP in billions of chained 2009 dollars'].diff()
    gdp['signal'] = gdp['difference_2009'] < 0
    gdp['signal'] = gdp['signal'].apply(lambda z:1 if z else 0)


    # Boolean indexers for recession start and stops.
    rec_start = (gdp['signal'] == 1) & (gdp['signal'].shift(-1) == 1)
    rec_end = (gdp['signal'] == 0) & (gdp['signal'].shift(-1) == 0)

    # Mark the recession start/stops as True/False.
    gdp.loc[rec_start, 'recession'] = True
    gdp.loc[rec_end, 'recession'] = False

    # Forward fill the recession column with the last known Boolean.
    # Fill any NaN's as False (i.e. locations before the first start/stop).
    gdp['recession'] = gdp['recession'].ffill().fillna(False)
    
    return gdp[(gdp['recession'] == False) & 
               (gdp['CYQ'] > get_recession_start())]['CYQ'].values[1]


'2009q4'

In [106]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4, 5, 6])
    gdp.columns= ['CYQ', 'GDP in billions of current dollars',
                  'GDP in billions of chained 2009 dollars']

    # extracting quarters from 2000 and on
    gdp = gdp[gdp['CYQ'].str[:4].astype(int)>= 2000]

    gdp['difference_2009'] = gdp['GDP in billions of chained 2009 dollars'].diff()
    gdp['signal'] = gdp['difference_2009'] < 0
    gdp['signal'] = gdp['signal'].apply(lambda z:1 if z else 0)


    # Boolean indexers for recession start and stops.
    rec_start = (gdp['signal'] == 1) & (gdp['signal'].shift(-1) == 1)
    rec_end = (gdp['signal'] == 0) & (gdp['signal'].shift(-1) == 0)

    # Mark the recession start/stops as True/False.
    gdp.loc[rec_start, 'recession'] = True
    gdp.loc[rec_end, 'recession'] = False

    # Forward fill the recession column with the last known Boolean.
    # Fill any NaN's as False (i.e. locations before the first start/stop).
    gdp['recession'] = gdp['recession'].ffill().fillna(False)
    
    min_gdp = gdp[gdp['recession'] == True]['GDP in billions of chained 2009 dollars'].min()
    
    return gdp[gdp['GDP in billions of chained 2009 dollars'] == min_gdp]['CYQ'].values[0]

'2009q2'

In [195]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resul
    ting dataframe should have 67 columns, and 10,730 rows.
    '''
    house = pd.read_csv('City_Zhvi_AllHomes.csv')
    df = house.copy().drop(['RegionID', 'RegionName', 'State',
                            'Metro', 'CountyName', 'SizeRank'],
                           axis=1)

    df = (df.groupby(pd.PeriodIndex(df.columns, freq='Q'), axis=1)
          .mean()
          .iloc[:, 15:])
    # df['RegionName'] = house['RegionName']

    df = pd.concat([house['State'], house['RegionName'], df], axis=1)
    
    
    df['State'] = df['State'].map(states)
    df.set_index(['State', 'RegionName'], inplace=True)
    df.columns = ['2000q1',
                 '2000q2',
                 '2000q3',
                 '2000q4',
                 '2001q1',
                 '2001q2',
                 '2001q3',
                 '2001q4',
                 '2002q1',
                 '2002q2',
                 '2002q3',
                 '2002q4',
                 '2003q1',
                 '2003q2',
                 '2003q3',
                 '2003q4',
                 '2004q1',
                 '2004q2',
                 '2004q3',
                 '2004q4',
                 '2005q1',
                 '2005q2',
                 '2005q3',
                 '2005q4',
                 '2006q1',
                 '2006q2',
                 '2006q3',
                 '2006q4',
                 '2007q1',
                 '2007q2',
                 '2007q3',
                 '2007q4',
                 '2008q1',
                 '2008q2',
                 '2008q3',
                 '2008q4',
                 '2009q1',
                 '2009q2',
                 '2009q3',
                 '2009q4',
                 '2010q1',
                 '2010q2',
                 '2010q3',
                 '2010q4',
                 '2011q1',
                 '2011q2',
                 '2011q3',
                 '2011q4',
                 '2012q1',
                 '2012q2',
                 '2012q3',
                 '2012q4',
                 '2013q1',
                 '2013q2',
                 '2013q3',
                 '2013q4',
                 '2014q1',
                 '2014q2',
                 '2014q3',
                 '2014q4',
                 '2015q1',
                 '2015q2',
                 '2015q3',
                 '2015q4',
                 '2016q1',
                 '2016q2',
                 '2016q3']
    return df

In [196]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    frame = convert_housing_data_to_quarters().copy()
    frame = frame.loc[:,'2008q3':'2009q2']
    frame = frame.reset_index()
    def price_ratio(row):
        return (row['2008q3'] - row['2009q2'])/row['2008q3']
    
    frame['up&down'] = frame.apply(price_ratio,axis=1)
    #uni frame 
    
    uni_town = get_list_of_university_towns()['RegionName']
    uni_town = set(uni_town)

    def is_university_town(row):
        if row['RegionName'] in uni_town:
            return 1
        else:
            return 0
    frame['is_uni'] = frame.apply(is_university_town,axis=1)
    
    
    not_uni = frame[frame['is_uni']==0].loc[:,'up&down'].dropna()
    is_uni  = frame[frame['is_uni']==1].loc[:,'up&down'].dropna()
    def better():
        if not_uni.mean() < is_uni.mean():
            return 'non-university town'
        else:
            return 'university town'
    pVal = list(ttest_ind(not_uni, is_uni))[1]
    answer = (True,pVal,better())
    return answer

NameError: name 'convert_housing_frame_to_quarters' is not defined